# 设计文档

## 文档说明

本文档包含多码率LDPC码的译码器的主要设计思路，以及实现细节，对于设计以及代码中的实现内的 trick 会着重提及，以保证该项目的可用性。

## 整体介绍

## 主要组成部分

这是一个经典的 QC-LDPC 码硬件译码器的实现，基于 cpp 的译码算法可以参考另一个仓库中的内容。在这个经典的LDPC码硬件译码器的实现中，我们会设计到如下部分:

计算模块：
+ CPU
+ VPU

控制模块：
+ controller

存储模块：
+ llr_ram








## CPU




In [ ]:
# 地址生成
base = "\n\
always @(posedge clk) begin\n\
	if (vpu_addr_ena == 1'b1)  begin\n\
		vpu_addr_{port} <= vpu_addr_{port} + 1'b1;\n\
	end\n\
	else\n\
		vpu_addr_{port} <= row_start_{port};\n\
end\
"

# for x in range(24):
# 	print(base.format(port=x))


base_cpu = "\n\
always @(posedge clk) begin\n\
	if (cpu_addr_ena == 1'b1)  begin\n\
		cpu_addr_{port} <= cpu_addr_{port} + 1'b1;\n\
	end\n\
	else\n\
		cpu_addr_{port} <= 0;\n\
end\
"

for x in range(24):
	print(base_cpu.format(port=x))

## VPU


## controller


## llr_ram

## do 文件模板

```do

# 创建库及映射库
vlib work
vmap work work

# 这里参数都是可选的 不加其实也可以
# vlog -incr -work work ../../ip/

vlog -incr -work work ../../rtl/ldpc_vpu.v
vlog -incr -work work ../../tb/tb_ldpc_vpu.v

# 固定格式，虽不知为何一定要用 -voptargs=+acc 但默认就是这种方式
# 不加的话代码跑不下去
vsim -voptargs=+acc work.tb_ldpc_vpu

# 添加波形
# 这里可以引入更加复杂的设置
add wave -radix decimal sim:/tb_ldpc_vpu/*
# add wave sim:/tb_ldpc_vpu/inst_ldpc_vpu/*

# 指定以何种进制显示
# Use binary, octal, decimal, signed, unsigned, hexadecimal, ascii or default. 默认二进制
# add wave -radix hex sim:/tb_ldpc_vpu/llr_all
# 指定以何种颜色显示
# add wave -color Yellow sim:/tb_ldpc_vpu/llr_out_0

# 选择需要加入的文件
do wave.do

view wave

run -all

```